In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/cleaned_df.csv')
df['POOFips'] = df['POOFips'].apply(lambda x:'0'+str(int(x)) if int(x)<10000 else str(int(x)))

In [ ]:
def rainfall_data_combined(m1,y1,m2,y2):
  df_new = pd.DataFrame(columns = ['FIPS','County','Rainfall','Month','Year'])
  total_months = 12*(y2-y1) + m2-m1
  period = 0
  for i in range(total_months):
    if m1>12:
      y1+=1
      m1=1
    site = 'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/mapping/110-pcp-'+str(y1) +  f"{m1:02}" +'-1.csv'
    data = pd.read_csv(site,sep= 'delimiter')
    
    for j in range(data.shape[0]-3):
      try:
        row_val = data.iloc[j+3][0].split(',')[:3]
        month = f"{m1:02}"
        year = str(y1)
        row_val.append(month)
        row_val.append(year)
        df_new.loc[period + j] = row_val
      except:
        continue
    period+=data.shape[0]
    m1+=1
  return df_new

In [ ]:
#df_rainfall = rainfall_data_combined(1,2019,1,2023)

In [ ]:
def weather_data_combined(m1, y1, m2, y2):
  df_new = pd.DataFrame(columns = ['FIPS','County','Max_Temp','Month','Year'])
  total_months = 12*(y2-y1) + (m2-m1)
  period = 0
  for i in range(total_months):
    if m1>12:
      y1+=1
      m1=1
    site = 'https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/mapping/110-tmax-'+str(y1)+f"{m1:02}" + '-1.csv'
    data = pd.read_csv(site,sep= 'delimiter')
    
    for j in range(data.shape[0]-3):
      try:
        row_val = data.iloc[j+3][0].split(',')[:3]
        month = f"{m1:02}"
        year = str(y1)
        row_val.append(month)
        row_val.append(year)

        df_new.loc[period + j] = row_val
      except:
        continue
    period+=data.shape[0]
    m1+=1
  return df_new
      


In [ ]:
#df_weather = weather_data_combined(1,2019,1,2023)

In [ ]:
#df_weather.to_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/max_temp_df.csv')

In [ ]:
df_max_temp = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/max_temp_df.csv').iloc[:,1:]

In [ ]:
df_weather = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/weather_data_2019_01_2023_01.csv').iloc[:,1:]

In [ ]:
df_weather.columns = ['POOFips','POOCounty','Temp','StartMonth','StartYear','Rainfall']
state_codes = {'AL': '01', 'AK': '02', 'AZ': '04', 'AR': '05',
               'CA': '06', 'CO': '08', 'CT': '09', 'DE': '10',
               'DC': '11', 'FL': '12', 'GA': '13', 'HI': '15',
               'ID': '16', 'IL': '17', 'IN': '18', 'IA': '19',
               'KS': '20', 'KY': '21', 'LA': '22', 'ME': '23',
               'MD': '24', 'MA': '25', 'MI': '26', 'MN': '27',
               'MS': '28', 'MO': '29', 'MT': '30', 'NE': '31',
               'NV': '32', 'NH': '33', 'NJ': '34', 'NM': '35',
               'NY': '36', 'NC': '37', 'ND': '38', 'OH': '39',
               'OK': '40', 'OR': '41', 'PA': '42', 'RI': '44',
               'SC': '45', 'SD': '46', 'TN': '47', 'TX': '48',
               'UT': '49', 'VT': '50', 'VA': '51', 'WA': '53',
               'WV': '54', 'WI': '55', 'WY': '56'}
df_weather['POOFips'] = df_weather['POOFips'].str.replace(r'([A-Z]{2})', lambda x: state_codes[x.group(1)], regex=True).str.replace('-','')
df_weather.head()

,POOFips,POOCounty,Temp,StartMonth,StartYear,Rainfall
0,01001,"""Autauga County""",47,1,2019,6.78
1,01003,"""Baldwin County""",51.8,1,2019,3.6
2,01005,"""Barbour County""",49,1,2019,5.86
3,01007,"""Bibb County""",45.2,1,2019,6.49
4,01009,"""Blount County""",43.7,1,2019,7.26


In [ ]:
df_wf_weather = pd.merge(df[df['StartYear']>2018],df_wf_weather,on = ['StartMonth','StartYear','POOFips'],how = 'left')
df_wf_weather.head()

,X,Y,ControlDateTime,DiscoveryAcres,FireCause,POOCounty,POOFips,POOState,StartMonth,StartYear,EndMonth,EndYear,Temp,Rainfall,Max_Temp
0,-87.348925,34.101946,2022/10/19 00:04:59+00,0.1,Human,Winston,01133,US-AL,10,2022,10.0,2022.0,58.5,3.33,72.6
1,-87.367355,34.314726,2021/01/25 13:55:00+00,1.0,Human,Lawrence,01079,US-AL,1,2021,1.0,2021.0,42.2,2.89,51.7
2,-85.402504,32.654173,NaN,179.0,Undetermined,Lee,01081,US-AL,11,2022,NaN,NaN,55,6.49,67
3,-85.892422,33.395450,2019/10/02 16:25:00+00,0.1,Human,Clay,01027,US-AL,10,2019,10.0,2019.0,66,5.96,77.2
4,-87.184715,32.823056,2022/11/01 18:00:00+00,5.0,Human,Perry,01105,US-AL,10,2022,11.0,2022.0,60.8,3.42,75.1


In [ ]:
df_wf_weather.to_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/combined_df.csv')

In [ ]:
df_max_temp.columns = ['POOFips','POOCounty','Max_Temp','StartMonth','StartYear']
df_max_temp['POOFips'] = df_max_temp['POOFips'].str.replace(r'([A-Z]{2})', lambda x: state_codes[x.group(1)], regex=True).str.replace('-','')
df_max_temp.head()

,POOFips,POOCounty,Max_Temp,StartMonth,StartYear
0,01001,"""Autauga County""",57.7,1,2019
1,01003,"""Baldwin County""",61.8,1,2019
2,01005,"""Barbour County""",59.6,1,2019
3,01007,"""Bibb County""",55.8,1,2019
4,01009,"""Blount County""",52.2,1,2019


In [ ]:
#df_wf_weather = pd.merge(df_weather.drop(columns = ['POOCounty']),df_max_temp.drop(columns = ['POOCounty']),on = ['StartMonth','StartYear','POOFips'],how = 'left')
df_wf_weather.head()

,X,Y,ControlDateTime,DiscoveryAcres,FireCause,POOCounty,POOFips,POOState,StartMonth,StartYear,EndMonth,EndYear,Temp,Rainfall,Max_Temp
0,-87.348925,34.101946,2022/10/19 00:04:59+00,0.1,Human,Winston,01133,US-AL,10,2022,10.0,2022.0,58.5,3.33,72.6
1,-87.367355,34.314726,2021/01/25 13:55:00+00,1.0,Human,Lawrence,01079,US-AL,1,2021,1.0,2021.0,42.2,2.89,51.7
2,-85.402504,32.654173,NaN,179.0,Undetermined,Lee,01081,US-AL,11,2022,NaN,NaN,55,6.49,67
3,-85.892422,33.395450,2019/10/02 16:25:00+00,0.1,Human,Clay,01027,US-AL,10,2019,10.0,2019.0,66,5.96,77.2
4,-87.184715,32.823056,2022/11/01 18:00:00+00,5.0,Human,Perry,01105,US-AL,10,2022,11.0,2022.0,60.8,3.42,75.1


In [ ]:
df_wf_weather['Total_Month'] = 12*(df_wf_weather['EndYear']-df_wf_weather['StartYear']) + (df_wf_weather['EndMonth']- df_wf_weather['StartMonth'])
df_wf_weather = df_wf_weather.drop(columns = ['EndMonth','EndYear','ControlDateTime','POOCounty'])
df_wf_weather.head()

,X,Y,DiscoveryAcres,FireCause,POOFips,POOState,StartMonth,StartYear,Temp,Rainfall,Max_Temp,Total_Month
0,-87.348925,34.101946,0.1,Human,01133,US-AL,10,2022,58.5,3.33,72.6,0.0
1,-87.367355,34.314726,1.0,Human,01079,US-AL,1,2021,42.2,2.89,51.7,0.0
2,-85.402504,32.654173,179.0,Undetermined,01081,US-AL,11,2022,55,6.49,67,NaN
3,-85.892422,33.395450,0.1,Human,01027,US-AL,10,2019,66,5.96,77.2,0.0
4,-87.184715,32.823056,5.0,Human,01105,US-AL,10,2022,60.8,3.42,75.1,1.0


In [ ]:
combined_df = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/combined_df.csv').iloc[:,1:]

In [ ]:
combined_df

,X,Y,DiscoveryAcres,FireCause,POOFips,POOState,StartMonth,StartYear,Temp,Rainfall,Max_Temp,Total_Month
0,-87.348925,34.101946,0.1,Human,1133,US-AL,10,2022,58.5,3.33,72.6,0.0
1,-87.367355,34.314726,1.0,Human,1079,US-AL,1,2021,42.2,2.89,51.7,0.0
2,-85.402504,32.654173,179.0,Undetermined,1081,US-AL,11,2022,55.0,6.49,67.0,NaN
3,-85.892422,33.395450,0.1,Human,1027,US-AL,10,2019,66.0,5.96,77.2,0.0
4,-87.184715,32.823056,5.0,Human,1105,US-AL,10,2022,60.8,3.42,75.1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
131103,-110.405072,43.204676,0.0,Natural,56035,US-WY,8,2020,60.3,0.16,78.7,1.0
131104,-106.637580,43.705415,0.0,Human,56019,US-WY,7,2021,72.4,0.51,87.6,0.0
131105,-107.968930,44.785266,1.0,Human,56003,US-WY,9,2020,57.9,0.73,73.8,0.0
131106,-110.276007,41.779465,0.0,Undetermined,56023,US-WY,7,2021,66.2,0.85,83.2,0.0


In [ ]:
combined_df.loc[combined_df['FireCause']=='Undetermined','FireCause'] = 'Unknown'

In [ ]:
county_center = pd.read_excel('/content/drive/Shareddrives/Datanators/Data/Processed_Data/us-county-boundaries.xlsx')

In [ ]:
county_center.head()

,Geo Point,STUSAB,ALAND,AWATER,POOFips
261,"33.8033347429, -87.2973573971",AL,2048686078,36754696,01127
2635,"32.7626642246, -87.6291176816",AL,1667917734,32412739,01065
1075,"34.8100783219, -86.9813747654",AL,1450277886,122329712,01083
751,"32.5349201866, -86.642749235",AL,1539602137,25706961,01001
2726,"32.5966480504, -86.1491465307",AL,1601768864,99957936,01051


In [ ]:
county_center = county_center.sort_values(by='STATEFP')

In [ ]:
county_center['STATEFP'] = county_center['STATEFP'].apply(lambda x:'0'+str(int(x)) if int(x)<10 else str(int(x)))
county_center['COUNTYFP'] = county_center['COUNTYFP'].apply(lambda x:'0'+str(int(x)) if int(x)<100 else str(int(x)))
county_center['COUNTYFP'] = county_center['COUNTYFP'].apply(lambda x:'00'+str(int(x)) if int(x)<10 else str(x))
county_center['POOFips'] = county_center['STATEFP'] + county_center['COUNTYFP']

In [ ]:
county_center = county_center.drop(columns = ['STATEFP','COUNTYFP','NAMELSAD'])

In [ ]:
combined_df['POOFips'] = combined_df['POOFips'].apply(lambda x:'0'+str(int(x)) if int(x)<10000 else str(int(x)))

In [ ]:
big_df = pd.merge(combined_df,county_center.drop(columns = ['STUSAB']), on = ['POOFips'],how = 'left')


In [ ]:
big_df['ALAND'] = big_df['ALAND'].apply(lambda x: x/1000000)
big_df['AWATER'] = big_df['AWATER'].apply(lambda x: x/1000000)


In [ ]:
big_df.head()

,X,Y,DiscoveryAcres,FireCause,POOFips,POOState,StartMonth,StartYear,Temp,Rainfall,Max_Temp,Total_Month,Geo Point,ALAND,AWATER,Geo_X,Geo_Y,Distance
0,-87.348925,34.101946,0.1,Human,01133,US-AL,10,2022,58.5,3.33,72.6,0.0,"34.1492251125, -87.3736834128",1587.614057,48.917203,-87.37,34.15,5.684563
1,-87.367355,34.314726,1.0,Human,01079,US-AL,1,2021,42.2,2.89,51.7,0.0,"34.5216520166, -87.311044513",1788.864251,68.682382,-87.31,34.52,23.423866
2,-85.402504,32.654173,179.0,Unknown,01081,US-AL,11,2022,55.0,6.49,67.0,NaN,"32.6011364841, -85.355562704",1573.514768,21.526461,-85.36,32.60,7.220082
3,-85.892422,33.395450,0.1,Human,01027,US-AL,10,2019,66.0,5.96,77.2,0.0,"33.2690858673, -85.8605470841",1564.251834,5.285207,-85.86,33.27,14.270924
4,-87.184715,32.823056,5.0,Human,01105,US-AL,10,2022,60.8,3.42,75.1,1.0,"32.6384656558, -87.2944007854",1863.900621,10.937769,-87.29,32.64,22.612160


In [ ]:
big_df['Geo_Y'] =  big_df['Geo Point'].apply(lambda x: round(float(x.split(',')[0]),2))
big_df['Geo_X'] = big_df['Geo Point'].apply(lambda x: round(float(x.split(',')[1]),2))


In [ ]:
def distance(lat1, lon1, lat2, lon2):
    # Earth radius in kilometers
    R = 6371

    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate the differences in latitude and longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Apply the haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    #Assuming the wildfires are usually in forest/hilly area, we multiply by a factor of 1.1*1.6 to get the distance in miles
    d = 1.1*R * c

    return d
big_df['Distance'] = big_df.apply(lambda x: distance(x['Y'],x['X'],x['Geo_Y'],x['Geo_X']),axis = 1)

In [ ]:
big_df = big_df[~(big_df['POOState']=='US-PR')]

In [ ]:
#big_df.to_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/big_df.csv')
big_df = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/big_df.csv').iloc[:,1:]

In [ ]:
#define a function to get the top 15 counties which reported the highest number of wildfires across multiple years and take their intersection
def freq_wfire(big_df,state,yr1 = 2019, yr2=2022):
  county_list = big_df[big_df['POOState']==state]['POOFips'].unique().tolist()
  for yr in range(yr1,yr2+1):
    c_list = big_df[(big_df['POOState']==state) &(big_df['StartYear']==yr)  ]['POOFips'].value_counts().nlargest(20).index.tolist()
    county_list = set(county_list).intersection(c_list)
  return list(county_list)

In [ ]:
#We look at the weighted centroid for each county over the months to realize the potential placement of a local fire station.
#Let's also do it state by state
#Pick states AZ,ID,MT,NM,OR,UT,WY
AZ_counties = big_df[big_df['POOState']=='US-AZ']['POOFips'].unique()
ID_counties = big_df[big_df['POOState']=='US-ID']['POOFips'].unique()
MT_counties = big_df[big_df['POOState']=='US-MT']['POOFips'].unique()
NM_counties = big_df[big_df['POOState']=='US-NM']['POOFips'].unique()
OR_counties = big_df[big_df['POOState']=='US-OR']['POOFips'].unique()
UT_counties = big_df[big_df['POOState']=='US-UT']['POOFips'].unique()
WY_counties = big_df[big_df['POOState']=='US-WY']['POOFips'].unique()

In [ ]:
def w_lon(big_df):
  l = big_df['X']
  w = big_df['DiscoveryAcres']
  return (l*w).sum()/w.sum()
def w_lat(big_df):
  l = big_df['Y']
  w = big_df['DiscoveryAcres']
  return (l*w).sum()/w.sum()  
avg_lon = big_df[big_df['POOFips']==AZ_counties[0]][['X','Y','DiscoveryAcres','StartYear','StartMonth']].groupby(['StartYear','StartMonth']).apply(w_lon).reset_index()
avg_lat = big_df[big_df['POOFips']==AZ_counties[0]][['X','Y','DiscoveryAcres','StartYear','StartMonth']].groupby(['StartYear','StartMonth']).apply(w_lat).reset_index()


<ipython-input-72-058b30f42d46>:4: RuntimeWarning: invalid value encountered in double_scalars
  return (l*w).sum()/w.sum()
<ipython-input-72-058b30f42d46>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (l*w).sum()/w.sum()


In [ ]:
tst_df = pd.merge(avg_lat,avg_lon,on = ['StartYear','StartMonth'], how = 'left')
import plotly.express as px
colors = ['#e6194B', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', 
          '#42d4f4', '#f032e6', '#bfef45', '#fabebe', '#469990', '#e6beff']
fig = px.scatter(tst_df, x ='0_x',y = '0_y',
                 color= 'StartMonth',color_continuous_scale=colors,
                 animation_frame = 'StartYear')
fig.update_coloraxes(colorbar_title='Index', colorbar_len=0.5, colorbar_thickness=20)
fig.update_layout(yaxis_range=[-109.91,-109],xaxis_range = [33.65,37.5])
fig.update_traces(marker=dict(size=15,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

In [ ]:
centers = []
for yr in range(2019,2023):
  kmeans = KMeans(n_clusters=2)
  kmeans.fit(tst_df[tst_df['StartYear']==yr].dropna()[['0_x','0_y']])
  centers.extend(kmeans.cluster_centers_)
centers

In [ ]:
import itertools

def max_distances(points):
    """
    Find the three pairs of points with the maximum distance between them.
    """
    # Create all combinations of pairs of points
    pairs = list(itertools.combinations(points, 2))

    # Calculate the distances between each pair of points
    distances = [np.linalg.norm(np.array(p1) - np.array(p2)) for (p1, p2) in pairs]

    # Find the indices of the pairs with the maximum distances
    max_indices = np.argsort(distances)[-2:]

    # Get the corresponding pairs of points
    max_pairs = [pairs[i] for i in max_indices]
    coords = list(itertools.chain(*max_pairs))
    nested_list = [item.tolist() for item in coords]
    unique_sublists = set(map(tuple, nested_list))
    unique_list = list(map(list, unique_sublists))
    return unique_list

max_distances(centers)

[[34.2915393432344, -109.51478284650324],
 [36.361629863363675, -109.18154785527192],
 [36.257855741834284, -109.40918177134019]]

In [ ]:
pop_data = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Table.csv').drop(columns =['GeoName']).dropna()

In [ ]:
pop_data.columns = ['POOFips', 'POP_2019','POP_2020','POP_2021','POP_2022']
pop_data['POOFips'] = pop_data['POOFips'].apply(lambda x:'0'+str(int(x)) if int(x)<10000 else str(int(x)))
pop_data.head()

,POOFips,POP_2019,POP_2020,POP_2021,POP_2022
0,01001,57728,58245,58877,59095
1,01003,221132,227079,233140,239294
2,01005,25380,25205,25180,24964
3,01007,22383,22405,22223,22477
4,01009,58765,58956,59081,59041


In [ ]:
#pop_data.to_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/pop_data.csv')
#pop_data = pd.read_csv('/content/drive/Shareddrives/Datanators/Data/Processed_Data/pop_data.csv')

In [ ]:
big_df['POOFips'] = big_df['POOFips'].apply(lambda x:'0'+str(int(x)) if int(x)<10000 else str(int(x)))
big_df = pd.merge(big_df,pop_data, on = ['POOFips'], how = 'left')


In [ ]:
min(big_df['ALAND'])

26.817671

In [ ]:
big_df.head()

NameError: ignored

In [ ]:
from plotly import animation
import plotly.express as px
# compute value counts and percentages
counts = big_df[big_df['StartYear']==2020]['FireCause'].dropna().value_counts()
percentages = counts / counts.sum()

# create pie chart
fig = px.pie( values=counts, names=counts.index, 
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(title={'text': "Cause of Wildland Fires", 'x': 0.49, 'y': 0.95},
                  showlegend = False)
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label'
)
#fig.write_html('Wildland_Fires.html')
# show plot
fig.show()


In [ ]:
counts['Human'] +=counts['Unknown']/2
counts['Natural']+=counts['Unknown']/2
counts2 = counts.iloc[1:]

# create pie chart
fig = px.pie(values =  counts2, names=counts2.index, 
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(title={'text': "Cause of wildfires, 2019-22", 'x': 0.494, 'y': 0.95},
                  showlegend = False)
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label'
)
#fig.write_html('Extrapolated_cause.html')
# show plot
fig.show()

In [ ]:
big_df.columns

Index(['X', 'Y', 'DiscoveryAcres', 'FireCause', 'POOFips', 'POOState',
       'StartMonth', 'StartYear', 'Temp', 'Rainfall', 'Max_Temp',
       'Total_Month', 'Geo Point', 'ALAND', 'AWATER', 'Geo_X', 'Geo_Y',
       'POP_2019', 'POP_2020', 'POP_2021', 'POP_2022'],
      dtype='object')

In [ ]:
wfire_freq_state = big_df['POOState'].value_counts()[:10]
import plotly.graph_objects as go

# Data

# Create figure
fig = go.Figure()

# Add trace
fig.add_trace(
    go.Bar(
        x=wfire_freq_state.index.tolist(),
        y=wfire_freq_state.values,
        text=wfire_freq_state.values,
        textposition='inside',
        marker_color='lightskyblue'
    )
)

# Remove y-axis
fig.update_yaxes(visible=False, showticklabels=False)
fig.update_xaxes(tickvals=[label for label in wfire_freq_state.index.tolist()],
                 ticktext = [label[3:] for label in fig.data[0].x])
fig.update_layout(title={
        'text': 'Wildfire Occurences by State',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'},
         plot_bgcolor='rgba(0,0,0,0)')

# Show figure
fig.show()


In [ ]:
big_df.groupby(['StartMonth', 'StartYear']).size().reset_index(name='count')
fig = px.bar(big_df.groupby(['StartMonth', 'StartYear']).size().reset_index(name='count'), x='StartMonth', y='count', animation_frame='StartYear',
        text='count')
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct','Nov','Dec']})
fig.update_xaxes(tickvals = list(range(1,13)),
              ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct','Nov','Dec'])
fig.update_layout(title={
        'text': 'Wildfire Occurences every month',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'},
         plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(yaxis_visible=False)
fig.show()

In [ ]:

fig = px.bar(big_df.groupby(['StartMonth', 'StartYear']).size().reset_index(name='count'), x='StartMonth', y='count', animation_frame='StartYear',
        text='count')
fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct','Nov','Dec']})
fig.update_xaxes(tickvals = list(range(1,13)),
              ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct','Nov','Dec'])
fig.update_layout(title={
        'text': 'Wildfire Occurences every month',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'},
         plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(yaxis_visible=False)
fig.show()

In [ ]:
big_df.columns

Index(['X', 'Y', 'DiscoveryAcres', 'FireCause', 'POOFips', 'POOState',
       'StartMonth', 'StartYear', 'Temp', 'Rainfall', 'Max_Temp',
       'Total_Month', 'Geo Point', 'ALAND', 'AWATER', 'Geo_X', 'Geo_Y',
       'POP_2019', 'POP_2020', 'POP_2021', 'POP_2022'],
      dtype='object')

In [ ]:
abcd = big_df.groupby(['StartYear','FireCause'])['FireCause'].count()
abcd

StartYear  FireCause
2019       Human         9647
           Natural       5066
           Unknown       9978
2020       Human        15526
           Natural       4483
           Unknown      13032
2021       Human        15125
           Natural       5787
           Unknown      15387
2022       Human        11091
           Natural       4819
           Unknown      21086
Name: FireCause, dtype: int64

In [ ]:
#abcd.loc[abcd.index.get_level_values('FireCause')!='Unknown']
fig = px.bar(abcd, x='StartYear', y='Counts',color = 'FireCause',
        text='Counts')

fig.update_layout(title={
        'text': 'Cause of wildfires over the years',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top'},
         plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout(yaxis_visible=False)
fig.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=[2019, 2020,2021,2022],
        ticktext=['2019', '2020', '2021','2022']
    )
)
fig.show()

In [ ]:
abcd = pd.DataFrame(abcd.loc[abcd.index.get_level_values('FireCause')!='Unknown'])
abcd.columns = ['Counts']

In [ ]:
abcd = abcd.reset_index()
abcd

,StartYear,FireCause,Counts
0,2019,Human,14636
1,2019,Natural,10055
2,2020,Human,22042
3,2020,Natural,10999
4,2021,Human,22818
5,2021,Natural,13480
6,2022,Human,21634
7,2022,Natural,15362


In [ ]:
abcd.loc[(slice(None),'Human')] = abcd.loc[(slice(None),'Human')].values + (abcd.loc[(slice(None),'Unknown')].values/2).astype(int)
abcd.loc[(slice(None),'Natural')] = abcd.loc[(slice(None),'Natural')].values + (abcd.loc[(slice(None),'Unknown')].values/2).astype(int)
